In [3]:
import requests
import osmnx as ox

# Function to fetch data from OpenWeather API
def fetch_weather_data(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        data['main']['temp'] = data['main']['temp'] - 273.15  # Convert Kelvin to Celsius
        return data
    else:
        print(f"Error fetching weather data: {response.status_code}")
        return None

# Function to fetch data from WAQI API
def fetch_aqi_data(city, api_key):
    url = f"http://api.waqi.info/feed/{city}/?token={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("status") == "ok":
            return data["data"]
        else:
            print(f"AQI API error: {data.get('data')}")
    else:
        print(f"Error fetching AQI data: {response.status_code}")
    return None

# Function to fetch route data from OpenStreetMap
def fetch_osm_route_data(origin_coords, destination_coords):
    """
    Fetch route data between two locations using OpenStreetMap.
    
    Args:
        origin_coords (tuple): Latitude and longitude of the starting point.
        destination_coords (tuple): Latitude and longitude of the ending point.
        
    Returns:
        route (list): List of latitude-longitude pairs for the route.
    """
    try:
        # Download the graph of the area
        G = ox.graph_from_point(origin_coords, dist=20000, network_type='drive')

        # Find the nearest nodes
        origin_node = ox.distance.nearest_nodes(G, origin_coords[1], origin_coords[0])
        destination_node = ox.distance.nearest_nodes(G, destination_coords[1], destination_coords[0])

        # Find the shortest path
        route = ox.shortest_path(G, origin_node, destination_node, weight='length')

        # Get route geometry
        route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
        return route_coords
    except Exception as e:
        print(f"Error fetching OSM data: {e}")
        return None

# Example API keys (Replace with your own)
weather_api_key = "9a939355165997c5bc96fe69ce3370cd"
aqi_api_key = "99e2255e97e295eea3d1d01735b9faf64ed8a4e1"

# Example usage
city = "Mumbai"
origin_coords = (19.0760, 72.8777)  # Mumbai
destination_coords = (19.1076, 72.8376)  # Andheri

# Fetch data
weather_data = fetch_weather_data(city, weather_api_key)
aqi_data = fetch_aqi_data(city, aqi_api_key)
osm_route_data = fetch_osm_route_data(origin_coords, destination_coords)

# Print fetched data
if weather_data:
    print("Weather Data:", weather_data)
if aqi_data:
    print("AQI Data:", aqi_data)
if osm_route_data:
    print("OSM Route Data (Coordinates):", osm_route_data)



Weather Data: {'coord': {'lon': 72.8479, 'lat': 19.0144}, 'weather': [{'id': 711, 'main': 'Smoke', 'description': 'smoke', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 24.99000000000001, 'feels_like': 297.92, 'temp_min': 298.09, 'temp_max': 298.14, 'pressure': 1014, 'humidity': 47, 'sea_level': 1014, 'grnd_level': 1013}, 'visibility': 2100, 'wind': {'speed': 1.54, 'deg': 120}, 'clouds': {'all': 0}, 'dt': 1737170672, 'sys': {'type': 1, 'id': 9052, 'country': 'IN', 'sunrise': 1737164681, 'sunset': 1737204772}, 'timezone': 19800, 'id': 1275339, 'name': 'Mumbai', 'cod': 200}
AQI Data: {'aqi': 164, 'idx': 7020, 'attributions': [{'url': 'http://worldweather.wmo.int', 'name': 'World Meteorological Organization - surface synoptic observations (WMO-SYNOP)'}, {'url': 'https://in.usembassy.gov/embassy-consulates/new-delhi/air-quality-data/', 'name': "U.S. Embassy and Consulates' Air Quality Monitor in India", 'logo': 'US-StateDepartment.png'}, {'url': 'https://waqi.info/', 'name': 'World

In [5]:
import numpy as np
import pickle
import requests
from geopy.geocoders import Nominatim

# Load the trained Random Forest model
with open(r'D:\My Data\Downloads\Aqi recommendation\Aqi recommendation\train_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

# API Keys (Please replace these with your actual keys)
WAQI_API_KEY = '99e2255e97e295eea3d1d01735b9faf64ed8a4e1'
OPENWEATHER_API_KEY = '9a939355165997c5bc96fe69ce3370cd'

# Rule-based logic for AQI insights
def rule_based_logic(aqi):
    if aqi >= 250:
        return "Hazardouds conditions detected! It's recommended to avoid outdoor activities. Also intallation of air purifiers is highly Recommended."
    elif aqi >= 200:
        return "Sever Conditions detected. Prolonged exposure can cause chronic health issues or organ damage. Caution is advised if you are sensitive to air quality."
    elif aqi >= 150:
        return "Unhealthy conditions outside. Make sure to wear masks!"
    elif aqi >= 100:
        return "Poor conditions, prople with respiratory issues may face problems."
    elif aqi >= 50:
        return "Moderate Conditions detected. Outdoor activities might cause mild discomfort in breathing."
    else:
        return "Air quality is good. Enjoy your outdoor activities safely."

# Function to fetch pollution data from WAQI API
def fetch_pollution_data(city):
    url = f'http://api.waqi.info/feed/{city}/?token=99e2255e97e295eea3d1d01735b9faf64ed8a4e1'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        aqi = data['data']['aqi']
        return aqi
    else:
        return None

# Function to fetch weather data from OpenWeather API
def fetch_weather_data(lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid=9a939355165997c5bc96fe69ce3370cd'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp'] - 273.15  # Convert from Kelvin to Celsius
        weather_description = data['weather'][0]['description']
        return f"Weather: {weather_description}, Temp: {temperature:.2f}°C"
    else:
        return "Error fetching weather data."
    
geolocator = Nominatim(user_agent="my_geopy_app")

# Function to get latitude and longitude from a place name
def get_coordinates(place_name):
    location = geolocator.geocode(place_name)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None


# Function to fetch traffic data from OpenStreetMap API
def get_traffic_data(start_location, end_location):
    url = f"https://router.project-osrm.org/route/v1/driving/{start_location[1]},{start_location[0]};{end_location[1]},{end_location[0]}?overview=full"
    response = requests.get(url)
    return response.json()

# Function to suggest safer routes
def suggest_safer_routes(start_place, end_place):
    start_coordinates = get_coordinates(start_place)
    end_coordinates = get_coordinates(end_place)

    if not start_coordinates or not end_coordinates:
        return "Could not find coordinates for one of the locations."

    traffic_data = get_traffic_data(start_coordinates, end_coordinates)

    if traffic_data and 'routes' in traffic_data:
        routes = traffic_data['routes']
        recommendations = []
        for route in routes:
            distance = route['distance'] / 1000  # Convert to kilometers
            duration = route['duration'] / 60  # Convert to minutes
            recommendations.append(f"Route: {distance:.2f} km, Estimated time: {duration:.2f} minutes")
        return recommendations
    else:
        return ["No routes found."]

# Function to simulate the chatbot's response and choices
def chatbot_response():
    city = "Mumbai"
    aqi = fetch_pollution_data(city)
    
    if aqi is None:
        return "Error fetching pollution data."
    
    # Get insights based on AQI
    insights = rule_based_logic(aqi)
    print(f"Today's AQI in {city}: {aqi}. {insights}")

    # User choices
    print("\nWhat would you like to do?")
    print("1. Get detailed weather data")
    print("2. See the latest pollution statistics")
    print("3. Know safer routes to travel")
    
    user_choice = input("Please enter the number of your choice: ")
    
    if user_choice == '1':
        lat, lon = 19.0760, 72.8777  # Mumbai coordinates
        weather_info = fetch_weather_data(lat, lon)
        print(weather_info)
        
    elif user_choice == '2':
        print(f"Latest pollution statistics for {city}: AQI: {aqi}")
        
    elif user_choice == '3':
        print("Welcome to the Travel Assistant!")
        start_place = input("Please enter your starting location: ")
        end_place = input("Please enter your destination: ")

        safer_routes = suggest_safer_routes(start_place, end_place)
        print("Safer route recommendations:")
        for route in safer_routes:
            print(route)
    else:
        print("Invalid choice. Please select a valid option.")

if __name__ == "__main__":
    chatbot_response()

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Today's AQI in Mumbai: 164. Unhealthy conditions outside. Make sure to wear masks!

What would you like to do?
1. Get detailed weather data
2. See the latest pollution statistics
3. Know safer routes to travel
Latest pollution statistics for Mumbai: AQI: 164


1